In [1]:
%load_ext dotenv
%dotenv

In [2]:
from datetime import datetime
import pandas as pd
from ch07_tools import get_answer
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    context_recall,
    faithfulness,
)


from tqdm import tqdm
from utils import neo4j_driver


/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
get_answer("Who acted in the most movies?")

('The actor who acted in the most movies is Tom Hanks, with a movie count of 12.',
 [{'role': 'assistant',
   'content': 'For the question: \'Which actor has acted in the most movies?\', we have the answer: \'[[{"actor": "Tom Hanks", "movie_count": 12}]]\''},
  {'role': 'assistant',
   'content': 'For the question: \'Who is the actor with the highest number of movie roles?\', we have the answer: \'[[{"actor": "Tom Hanks", "roles_count": 12}]]\''}])

In [4]:
test_data = pd.read_csv("../data/benchmark_data.csv", delimiter=";")
test_data

,question,cypher
0,Hello,"RETURN ""greeting and reminder it can only answ..."
1,What can you do?,"RETURN ""answer questions related to movies and..."
2,How many movies are there?,MATCH (m:Movie) RETURN count(*) AS movies
3,Who acted in the most movies?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie)\nRETUR...
4,Which movies released between 1990 and 2000 ha...,MATCH (m:Movie) WHERE m.released >= 1990 AND m...
5,Who has the longest name among all actors?,MATCH (p:Person)-[:ACTED_IN]->(:Movie) RETURN ...
6,List people born before 1940.,MATCH (p:Person) WHERE p.born < 1940 RETURN p....
7,Who was born in 1965 and has directed a movie?,MATCH (p:Person)-[:DIRECTED]->(m:Movie) WHERE ...
8,Who acted in Top Gun?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:...
9,"List the movies with a title starting with ""The""","MATCH (m:Movie) WHERE m.title STARTS WITH ""The..."


In [5]:
answers = []
ground_truths = []
latencies = []
contexts = []

for i, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing rows"):
    ground_truth, _, _ = neo4j_driver.execute_query(row["cypher"])
    ground_truths.append([str(el.data()) for el in ground_truth])
    start = datetime.now()
    try:
        answer, context = get_answer(row["question"])
        context = [el['content'] for el in context]
    except Exception:
        answer, context = None, []
    latencies.append((datetime.now() - start).total_seconds())
    answers.append(answer)
    contexts.append(context)

Processing rows: 100%|██████████| 13/13 [03:20<00:00, 15.45s/it]


In [6]:
test_data['ground_truth'] = [str(el) for el in ground_truths]
test_data['answer'] = answers
test_data['latency'] = latencies
test_data['retrieved_contexts'] = contexts

In [7]:
dataset = Dataset.from_pandas(test_data.fillna("I don't know"))
result = evaluate(
    dataset,
    metrics=[
        answer_correctness,
        context_recall,
        faithfulness,
    ],
)
print(result)

Evaluating:   0%|          | 0/39 [00:00<?, ?it/s]

{'answer_correctness': 0.7153, 'context_recall': 0.6923, 'faithfulness': 1.0000}


In [8]:
for key in ["answer_correctness", "context_recall", "faithfulness"]:
    test_data[key] = [el[key] for el in result.scores]
test_data

,question,cypher,ground_truth,answer,latency,retrieved_contexts,answer_correctness,context_recall,faithfulness
0,Hello,"RETURN ""greeting and reminder it can only answ...","['{\'""greeting and reminder it can only answer...",I can only answer questions about movies and t...,5.876216,[For the question: 'Which actor has acted in t...,0.196927,0.0,1.0
1,What can you do?,"RETURN ""answer questions related to movies and...","['{\'""answer questions related to movies and t...",I can provide information about movies and the...,9.577331,[For the question: 'Which actor has acted in t...,0.702383,0.0,1.0
2,How many movies are there?,MATCH (m:Movie) RETURN count(*) AS movies,"[""{'movies': 38}""]",There are 38 movies in total.,12.843184,[For the question: 'Which actor has acted in t...,0.957427,1.0,1.0
3,Who acted in the most movies?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie)\nRETUR...,"[""{'p.name': 'Tom Hanks', 'movieCount': 12}""]",The actor who has acted in the most movies is ...,22.990381,[For the question: 'Which actor has acted in t...,0.701865,1.0,1.0
4,Which movies released between 1990 and 2000 ha...,MATCH (m:Movie) WHERE m.released >= 1990 AND m...,"[""{'m.title': 'The Matrix', 'm.released': 1999...",The movie released between 1990 and 2000 that ...,64.656445,[For the question: 'Which actor has acted in t...,0.944827,1.0,1.0
5,Who has the longest name among all actors?,MATCH (p:Person)-[:ACTED_IN]->(:Movie) RETURN ...,"[""{'name': 'Philip Seymour Hoffman'}""]",The information needed to answer the question ...,11.923005,[For the question: 'Which actor has acted in t...,0.183914,0.0,1.0
6,List people born before 1940.,MATCH (p:Person) WHERE p.born < 1940 RETURN p....,"[""{'p.name': 'Max von Sydow'}"", ""{'p.name': 'G...",The people born before 1940 are:\n\n- Max von ...,7.005498,[For the question: 'Which actor has acted in t...,0.947563,1.0,1.0
7,Who was born in 1965 and has directed a movie?,MATCH (p:Person)-[:DIRECTED]->(m:Movie) WHERE ...,"[""{'p.name': 'Lana Wachowski'}"", ""{'p.name': '...",The information provided does not specify whic...,10.669961,[For the question: 'Which actor has acted in t...,0.179119,1.0,1.0
8,Who acted in Top Gun?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:...,"[""{'p.name': 'Val Kilmer'}"", ""{'p.name': 'Meg ...","The actors who acted in ""Top Gun"" are Val Kilm...",3.962218,[For the question: 'Which actor has acted in t...,0.950264,1.0,1.0
9,"List the movies with a title starting with ""The""","MATCH (m:Movie) WHERE m.title STARTS WITH ""The...","[""{'m.title': 'The Birdcage'}"", ""{'m.title': '...","The movies with a title starting with ""The"" ar...",9.150836,[For the question: 'Which actor has acted in t...,0.953924,1.0,1.0
